In [3]:
%load_ext autoreload

In [314]:

import argparse
import time
import os
import csv
from datetime import datetime
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torch.optim as optim
import torch.optim.lr_scheduler
from torchvision import datasets, transforms
from torch.autograd import Variable

import Levenshtein as Lev
from e14.main import *
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

parser = argparse.ArgumentParser(description='PyTorch RNN/LSTM Language Model')
parser.add_argument('--data-dir', type=str, default='./data',
                    help='location of the data corpus')
parser.add_argument('--weights-dir', type=str, default='./weights/',
                    help='data directory')
parser.add_argument('--lr', type=float, default=1e-3,
                    help='initial learning rate')
parser.add_argument('--clip', type=float, default=0.25, #default=0.25,
                    help='gradient clipping')
parser.add_argument('--epochs', type=int, default=50,
                    help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=32, metavar='N',
                    help='batch size')
parser.add_argument('--seed', type=int, default=2222,
                    help='random seed')
parser.add_argument('--cuda', action='store_false',
                    help='use CUDA')
parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                    help='report interval')
parser.add_argument('--wdecay', type=float, default=1.2e-6, #default=1.2e-6,
                    help='weight decay applied to all weights')
args = parser.parse_args("")
args.cuda = torch.cuda.is_available() and args.cuda
args.batch_size=1
# args.cuda = False

%autoreload

In [300]:
import numpy as np

class Node(object):
    def __init__(self, parent, state, value, cost, extras):
        super(Node, self).__init__()
        self.value = value
        self.parent = parent # parent Node, None for root
        self.state = state # recurrent layer hidden state
        self.cum_cost = parent.cum_cost + cost if parent else cost # e.g. -log(p) of sequence up to current node (including)
        self.length = 1 if parent is None else parent.length + 1
        self.extras = extras # can hold, for example, attention weights
        self._sequence = None

    def to_sequence(self):
        # Return sequence of nodes from root to current node.
        if not self._sequence:
            self._sequence = []
            current_node = self
            while current_node:
                self._sequence.insert(0, current_node)
                current_node = current_node.parent
        return self._sequence

    def to_sequence_of_values(self):
        return [s.value for s in self.to_sequence()]

    def to_sequence_of_extras(self):
        return [s.extras for s in self.to_sequence()]

def beam_search(initial_state_function, generate_function, X, seq_sizes, end_id, beam_width=4, num_hypotheses=1, max_length=50):
    """Beam search for neural network sequence to sequence (encoder-decoder) models.
    :param initial_state_function: A function that takes X as input and returns state (2-dimensonal numpy array with 1 row
                                   representing decoder recurrent layer state - currently supports only one recurrent layer).
    :param generate_function: A function that takes X, Y_tm1 (1-dimensional numpy array of token indices in decoder vocabulary
                              generated at previous step) and state_tm1 (2-dimensonal numpy array of previous step decoder recurrent
                              layer states) as input and returns state_t (2-dimensonal numpy array of current step decoder recurrent
                              layer states), p_t (2-dimensonal numpy array of decoder softmax outputs) and optional extras
                              (e.g. attention weights at current step).
    :param X: List of input token indices in encoder vocabulary.
    :param start_id: Index of <start sequence> token in decoder vocabulary.
    :param end_id: Index of <end sequence> token in decoder vocabulary.
    :param beam_width: Beam size. Default 4.
    :param num_hypotheses: Number of hypotheses to generate. Default 1.
    :param max_length: Length limit for generated sequence. Default 50.
    """
#     if isinstance(X, list) or X.ndim == 1:
#         X = np.array([X], dtype=np.int32).T
#     assert X.ndim == 2 and X.shape[1] == 1, "X should be a column array with shape (input-sequence-length, 1)"
    
    initial_state, initial_value = initial_state_function(X, seq_sizes)
    next_fringe = [Node(parent=None, state=initial_state, value=initial_value, cost=0.0, extras=None)]
    hypotheses = []

    for step in range(max_length):
        fringe = []
#         print("step", step, "next_fringe", next_fringe)
        for n in next_fringe:
            if (step != 0 and n.value == end_id) or step == max_length - 1:
                hypotheses.append(n)
            else:
                fringe.append(n)

#         print("step", step, "fringe", fringe)
        if not fringe or len(hypotheses) >= num_hypotheses:
            break

        Y_tm1 = [n.value for n in fringe]
        state_tm1 = [n.state for n in fringe]
        state_t, p_t, extras_t = generate_function(X, Y_tm1, state_tm1)
        Y_t = np.argsort(p_t, axis=1)[:,-beam_width:] # no point in taking more than fits in the beam
        next_fringe = []
        for Y_t_n, p_t_n, extras_t_n, state_t_n, n in zip(Y_t, p_t, extras_t, state_t, fringe):
            Y_nll_t_n = -np.log(p_t_n[Y_t_n])
            
            for y_t_n, y_nll_t_n in zip(Y_t_n, Y_nll_t_n):
                n_new = Node(parent=n, state=state_t_n, value=y_t_n, cost=y_nll_t_n, extras=extras_t_n)
                next_fringe.append(n_new)

        next_fringe = sorted(next_fringe, key=lambda n: n.cum_cost)[:beam_width] # may move this into loop to save memory

    hypotheses.sort(key=lambda n: n.cum_cost)
    return hypotheses[:num_hypotheses]


In [38]:
def get_data_loaders(args, use_dev_for_train=False):
    print("loading data")

    xdev = np.load(args.data_dir + '/dev.npy')
    ydev = np.load(args.data_dir + '/dev_transcripts.npy')
    if use_dev_for_train:
        xtrain = np.load(args.data_dir + '/dev.npy')
        ytrain = np.load(args.data_dir + '/dev_transcripts.npy')
    else:
        xtrain = np.load(args.data_dir + '/train.npy')
        ytrain = np.load(args.data_dir + '/train_transcripts.npy')

    print("load complete")
    kwargs = {'num_workers': 3, 'pin_memory': True} if args.cuda else {}
    train_loader = torch.utils.data.DataLoader(
        MyDataset(xtrain, ytrain),
        batch_size=args.batch_size, shuffle=True, collate_fn=my_collate, **kwargs)
    dev_loader = torch.utils.data.DataLoader(
        MyDataset(xdev, ydev),
        batch_size=args.batch_size, shuffle=True, collate_fn=my_collate, **kwargs)

    return train_loader, dev_loader

train_loader, dev_loader = get_data_loaders(args, True)

loading data
load complete
n_utters 1139 total_labels 111376
n_utters 1139 total_labels 111376


In [311]:
def decode_random_search(model, frames, seq_sizes, N=3, max_len=250):
    def get_gen_len(gen):
        for idx, c in enumerate(gen):
            if c == chr2idx['<eos>']:
                return idx + 1
        return len(chrs)
    
    random_gens = []
    criterion = SeqCrossEntropyLoss()
    for n in range(N):
        state, pred = model.get_initial_state(frames, seq_sizes)
        state = [state]
        pred = [pred]
        gen = []
        gen_probs = []
        for i in range(max_len):
            state, probs, extras = model.generate(frames, pred, state)
            ci = torch.multinomial(to_tensor(probs[0]), 1).squeeze().numpy()[0]
            pred = [ci]
            gen.append(ci)
            gen_probs += probs
            if ci == chr2idx["<eos>"]:
                break

        all_labels = torch.zeros(1, max_len).long()
        for i, l in enumerate(gen):
            all_labels[0, i] = int(l)
        loss = criterion(Variable(to_tensor(np.array(gen_probs)).unsqueeze(0)),
                               torch.from_numpy(np.array([get_gen_len(gen)])),
                                Variable(all_labels)).data[0]
        random_gens.append((gen, loss))
    
    random_gens.sort(key=lambda x: x[1])
#     print(random_gens)
#     print(label_list_to_str([g[0] for g in random_gens]))
    return label_list_to_str([g[0][:-1] for g in random_gens])
    

def decode2(model, frames, seq_sizes):
    hypos = beam_search(model.get_initial_state, model.generate, frames, seq_sizes,
                          chr2idx["<eos>"], beam_width=3, num_hypotheses=1, max_length=250)
    output = [n.to_sequence_of_values()[1:][:-1] for n in hypos]
    attentions = [n.to_sequence_of_extras() for n in hypos]
    return label_list_to_str(output), attentions

def decode(probs):
    # probs: FloatTensor (B, L, C)
    out = []
    for prob in probs:
        s = []
        for step in prob:
            #             idx = torch.multinomial(step, 1)[0]
            idx = step.max(0)[1][0]
            c = idx2chr[idx]
            if c == '<eos>':
                break
            s.append(c)
            
        out.append("".join(s))
    return out


In [7]:
def show_attentions(frames, attentions):
    bn = nn.BatchNorm1d(40)
    frames_img = bn(frames.squeeze()).data.numpy().T
    fig = plt.figure(figsize=(15, 10))
    plt.subplot(211)
    plt.imshow(frames_img, cmap='gray', interpolation='nearest', aspect='auto')
    
    atten_data = [a.data.cpu().numpy() for a in attentions[0] if a is not None]
    atten_data = np.concatenate(atten_data, axis=0)
    plt.subplot(212)
    plt.imshow(atten_data, cmap='gray', interpolation='nearest', aspect='auto')

In [8]:

def get_test_data_loaders(args):
    print("loading data")
    xtest = np.load(args.data_dir + '/test.npy')
    print("load complete")
    kwargs = {'pin_memory': True} if args.cuda else {}
    test_loader = torch.utils.data.DataLoader(
        MyDataset(xtest, None),
        batch_size=1, shuffle=False, collate_fn=my_collate, **kwargs)
    return test_loader
test_loader = get_test_data_loaders(args)

loading data
load complete
n_utters 268 total_labels -1


In [313]:
def predict2(args, csv_fpath, weights_fpath):
    model = LAS()
    model.load_state_dict(torch.load(weights_fpath))
    if args.cuda:
        model.cuda()

    model.eval()
#     test_loader = get_test_data_loaders(args)
#     test_loader = dev_loader
#     test_loader = train_loader
    softmax = nn.Softmax()
    #if True:
    with open(csv_fpath, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['Id', 'Predicted'])
        writer.writeheader()
        cnt = 0
        for batch, (frames, seq_sizes, labels, label_sizes) in enumerate(test_loader):
            batch += 1
            if args.cuda:
                frames = frames.cuda()
            frames = Variable(frames, volatile=True)
            
#             outputs, attentions = model(frames, Variable(labels), max_iters=250)
#             decoded0 = decode(softmax(outputs.squeeze()).unsqueeze(0).data.cpu())
#             print("decoded0[0]\n", decoded0[0])
            
#             outputs, attentions = model(frames, seq_sizes, None, max_iters=250)
#             decoded1 = decode(softmax(outputs.squeeze()).unsqueeze(0).data.cpu())
#             print("decoded1[0]\n", decoded1[0])
#             decoded = decoded1
            
#             decoded, attentions = decode2(model, frames, seq_sizes)
#             show_attentions(frames, attentions)

            decoded = decode_random_search(model, frames, seq_sizes, N=100)[:10]
            print(decoded)
            
            labels_str = labels2str(labels, label_sizes)  # [label2str(label) for label in to_np(labels)]
            print("\nLABEL:" + labels_str[0])
            for m in decoded:
                print("\nMINE: " + m + "\n")
            for s in decoded:
#                 if cnt % args.log_interval * 20 == 0:
#                     print(cnt, s)
                writer.writerow({"Id": cnt, "Predicted": s})
                cnt += 1
            break
    print("done")
    
# predict2(args, "./e9/garbage", "./e9/001_81.6819.w")
# predict2(args, "./e9/garbage", "./e9/006_76.2355.w")
# predict2(args, "./e9/garbage", "./e9/011_74.8967.w")
# predict2(args, "./e9/garbage", "./e9/016_73.0983.w")
# predict2(args, "./e9/garbage", "./e9/021_73.2393.w")

predict2(args, "./e13/garbage", "./e14/004_77.7879.w")


['TRAVEL LAST MONTH ARE GREENED PRRINCIPAL TO SHELTER HIS ENGLAND WATER TRANSPORTATION STAVE', 'TRAVEL LAST MONTH AGREED IN PRINCIPLE TO FELLOW ITS INLWANT WATER TRANSPORTATION STABID DORRING AND PIPE VABURAT', 'TRAVEL LAST MONTH AGREEDED PRINCIPLE TO SHELL ITS INLON WATER TRANSPORTATION STAN OF DOORING AND PIPE ADVOIATCA SHREAUSED S', 'TRAVEL LAST MONTH AGREED IN PRINCIPLE TO HELIT IN LETTER TRANSPORTATION STAVED IRON AND PIPE FABRICATION BUSINESSES FOR AN UNDISCLOSED SOME', 'TRAVEL LAST MONTH AGREED IN PRINCIPLE TO SHELTER IN LTORTO TRANSPORTATION STAVEEVER IN AN PIPE FAARCATION BUSINESSES FOR AN UNDISCLOSED SOME', 'TRAVEL LAST MONTH AGREED IN PRINCIPLE TO SHELTERS IN LOND WATER TRANSPORTATION STATED DAARING AND PRIVATATION BUSINESSES FOR AN UNDISCLOSED SOME', 'TRAVEL LAST MONTH AGREED IN PRINCIPLE TO FELL ITS  INLEN WATER TRANSPORTATION STATE DERIORING AND PIPE FABRICATION BUSINESSES FOR AN UNDISCLOSED', 'TRAVEL LAST MONTH ARREEDING PRINCIPLE TO SHELL ITS INLAND WATER TRANSPORTATION

In [279]:
a = torch.from_numpy(np.array([3]))
print(a)
# a = a.unsqueeze(0)
print(a)
for ai in a:
    print(ai)


 3
[torch.LongTensor of size 1]


 3
[torch.LongTensor of size 1]

3


In [73]:
def predict3(args, csv_fpath, weights_fpath):
    model = LAS()
    model.load_state_dict(torch.load(weights_fpath))
    if args.cuda:
        model.cuda()

    model.eval()
    test_loader = get_test_data_loaders(args)
    softmax = nn.Softmax()
    with open(csv_fpath, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['Id', 'Predicted'])
        writer.writeheader()
        cnt = 0
        for batch, (frames, seq_sizes, labels, label_sizes) in enumerate(test_loader):
            batch += 1
            if args.cuda:
                frames = frames.cuda()
            frames = Variable(frames, volatile=True)
            
#             outputs, attentions = model(frames, Variable(labels), max_iters=250)
#             decoded0 = decode(softmax(outputs.squeeze()).unsqueeze(0).data.cpu())
#             print("decoded0[0]\n", decoded0[0])
            
            outputs, attentions = model(frames, seq_sizes, None, max_iters=250)
            decoded1 = decode(softmax(outputs.squeeze()).unsqueeze(0).data.cpu())
            print("decoded1[0]\n     ", decoded1[0])
            decoded = decoded1
            
#             decoded, attentions = decode2(model, frames, seq_sizes)
            
#             show_attentions(frames, attentions)
            
            labels_str = labels2str(labels, label_sizes)  # [label2str(label) for label in to_np(labels)]
            print("--------", batch)
            print("\nLABEL:" + labels_str[0])
            for m in decoded:
                print("\nMINE: " + m + "\n")
            for s in decoded:
#                 if cnt % args.log_interval * 20 == 0:
#                     print(cnt, s)
                writer.writerow({"Id": cnt, "Predicted": s})
                cnt += 1
        
    print("done")
    

predict3(args, "./e14/e14_w04_greedy.csv", "./e14/004_77.7879.w")


loading data
load complete
n_utters 268 total_labels -1
decoded1[0]
      TRAVEL LAST MONTH AGREED IN PRINCIPLE TO SHELTER IT AND LED WATER TRANSPORTATION STAVED A DOORING AND PIPE FABRICATION BUSINESSES FOR AN UNDISCLOSED SOME
-------- 1

LABEL:_

MINE: TRAVEL LAST MONTH AGREED IN PRINCIPLE TO SHELTER IT AND LED WATER TRANSPORTATION STAVED A DOORING AND PIPE FABRICATION BUSINESSES FOR AN UNDISCLOSED SOME

decoded1[0]
      THE COMBINED BUSINESSES ACCOUNTED FOR TWO HUNDRED THIRTY FIVE MILLION DOLLARS OF DRAVIS EIGHT HUNDRED NINETY THREE MILLION DOLLARS IN NINETEEN EIGHTY FIVE REVENUE
-------- 2

LABEL:_

MINE: THE COMBINED BUSINESSES ACCOUNTED FOR TWO HUNDRED THIRTY FIVE MILLION DOLLARS OF DRAVIS EIGHT HUNDRED NINETY THREE MILLION DOLLARS IN NINETEEN EIGHTY FIVE REVENUE

decoded1[0]
      IN SEPTEMBER THE COMPANY RECEIVED NINETY SIX POINT EIGHT MILLION DOLLARS AS IT SHARE OF DAMAGES FROM A BREACH OF CONTRACT WARD RELATED TO A COAL PARTNERSHIP
-------- 3

LABEL:_

MINE: IN SEPTEMBER THE

decoded1[0]
      BUT THERE ARE ALSO GRISH
-------- 35

LABEL:_

MINE: BUT THERE ARE ALSO GRISH

decoded1[0]
      TWO STEP A FALL CAN MEAN HIGHER INFLATION AND EVENTUALLY HIGHER INTEREST RATES
-------- 36

LABEL:_

MINE: TWO STEP A FALL CAN MEAN HIGHER INFLATION AND EVENTUALLY HIGHER INTEREST RATES

decoded1[0]
      MR. ZAZI TO HIS IRAQ FLINK TO THE U. S. IS A SPECIALLY SENSITIVE ABOUT THE AMERICAN DECEPTION OVER IRAN AND DECEPTION OVER IRAN
-------- 37

LABEL:_

MINE: MR. ZAZI TO HIS IRAQ FLINK TO THE U. S. IS A SPECIALLY SENSITIVE ABOUT THE AMERICAN DECEPTION OVER IRAN AND DECEPTION OVER IRAN

decoded1[0]
      ASSOCIATED IN IN THEN IS AIR COLOR IS SUGGENERAL PARTNER THERE COLON HOW TELEPHONE HER IN HAS AND HAS A ONE PERCENT GENERAL PARTNER INTEREST IN THE PARTNERSHIP
-------- 38

LABEL:_

MINE: ASSOCIATED IN IN THEN IS AIR COLOR IS SUGGENERAL PARTNER THERE COLON HOW TELEPHONE HER IN HAS AND HAS A ONE PERCENT GENERAL PARTNER INTEREST IN THE PARTNERSHIP

decoded1[0]
      AND AIRCAL

decoded1[0]
      IT MAY BE THAT OUR STRUCTURE AND MULTIPLE SAID THE OF CORPORATIONS ISN'T THE MOST EFFECTIVE FOR THE FUTURE
-------- 70

LABEL:_

MINE: IT MAY BE THAT OUR STRUCTURE AND MULTIPLE SAID THE OF CORPORATIONS ISN'T THE MOST EFFECTIVE FOR THE FUTURE

decoded1[0]
      NO FIRM PLAN HAS BEEN DEVISED BUT IT IS UNDER CONSIDERATION TO REVIEW THE WHOLE STRUCTURE HE SAID
-------- 71

LABEL:_

MINE: NO FIRM PLAN HAS BEEN DEVISED BUT IT IS UNDER CONSIDERATION TO REVIEW THE WHOLE STRUCTURE HE SAID

decoded1[0]
      MR. ROBERTSON CAN BEST BE UNDERSTOOD NOT IS A NEW FIGURE IN THE CONSERVATIVE MOVEMENT BUT AS THE AIR OF THE POPULISTS AND FATHER CONFLINE
-------- 72

LABEL:_

MINE: MR. ROBERTSON CAN BEST BE UNDERSTOOD NOT IS A NEW FIGURE IN THE CONSERVATIVE MOVEMENT BUT AS THE AIR OF THE POPULISTS AND FATHER CONFLINE

decoded1[0]
      AS THE POPULISTS DIDN'T HAPPEAL TO LESS EDUCATED LOWER MINILACLASS PEOPLE WHO FIND ECONOMIC AND SOCIAL CHANGE FRIDAY MAINTENANCE SOCIAL CHANGE FRIDAY MAINT

decoded1[0]
      INSTEAD IT FOUND THAT A FASHION DESIGNED FIRM MEGAL CRUISE INTERNATIONAL WAS OPERATING IT ALLOCATION REGISTERED IS THAT OF MR. POEHL'S INVESTMENT FIRM
-------- 103

LABEL:_

MINE: INSTEAD IT FOUND THAT A FASHION DESIGNED FIRM MEGAL CRUISE INTERNATIONAL WAS OPERATING IT ALLOCATION REGISTERED IS THAT OF MR. POEHL'S INVESTMENT FIRM

decoded1[0]
      MR. POLLOW ALSO HOADS THE FASHION COMPANY
-------- 104

LABEL:_

MINE: MR. POLLOW ALSO HOADS THE FASHION COMPANY

decoded1[0]
      AT FORTY YEARS OLD MR. WAYNE IS RELATIVELY YOUNG FOR HIS JOB AND THAT OF SEPS COLLEAGUES WHO ARE USED TO A GRADUAL PROMOTION SYSTEM BASED ON SENIORITY
-------- 105

LABEL:_

MINE: AT FORTY YEARS OLD MR. WAYNE IS RELATIVELY YOUNG FOR HIS JOB AND THAT OF SEPS COLLEAGUES WHO ARE USED TO A GRADUAL PROMOTION SYSTEM BASED ON SENIORITY

decoded1[0]
      SO MR. WRAYNE TELLS PEOPLE USED FIFTY
-------- 106

LABEL:_

MINE: SO MR. WRAYNE TELLS PEOPLE USED FIFTY

decoded1[0]
      HIS M. B. A. ALSO WERE EXC

decoded1[0]
      NO FARM PLAN HAS BEEN DEVISED BUT IT HAS UNDER CONSIDERATION TO REVIEW THE WHOLE STRETCH IN HE SAID
-------- 133

LABEL:_

MINE: NO FARM PLAN HAS BEEN DEVISED BUT IT HAS UNDER CONSIDERATION TO REVIEW THE WHOLE STRETCH IN HE SAID

decoded1[0]
      THE PHILIPPINE MILITARY SAID IT HAD AN COVER A PLATTE BY COMMUNIST GOERALOS TO KILL RETIRE U. S. MAJOR GENERAL SINGLEBOUT
-------- 134

LABEL:_

MINE: THE PHILIPPINE MILITARY SAID IT HAD AN COVER A PLATTE BY COMMUNIST GOERALOS TO KILL RETIRE U. S. MAJOR GENERAL SINGLEBOUT

decoded1[0]
      THE PLAN WAS DISCOVERED AFTER A DECADING DATA CAPTURED AT LAST WOMEN SURRESTER TWENTY REBEL LEADERS
-------- 135

LABEL:_

MINE: THE PLAN WAS DISCOVERED AFTER A DECADING DATA CAPTURED AT LAST WOMEN SURRESTER TWENTY REBEL LEADERS

decoded1[0]
      THE REBELS OF ACCUSED SINGLE BUT FORMING ANTI COMMUNIST FIDULATIT GRIPS
-------- 136

LABEL:_

MINE: THE REBELS OF ACCUSED SINGLE BUT FORMING ANTI COMMUNIST FIDULATIT GRIPS

decoded1[0]
      EV

decoded1[0]
      HOWEVER IN LIGHT OF THE STAKES ALREADY HELD BY THE VARIOUS FACTIONS ONLY ABOUT TEN PERCENT OF GENERALE DOUBLE-QUOTE EACH SHARES APPEARED TO BE AVAILABLE ON THE MARKET
-------- 166

LABEL:_

MINE: HOWEVER IN LIGHT OF THE STAKES ALREADY HELD BY THE VARIOUS FACTIONS ONLY ABOUT TEN PERCENT OF GENERALE DOUBLE-QUOTE EACH SHARES APPEARED TO BE AVAILABLE ON THE MARKET

decoded1[0]
      THIS MEANS THAT FOR MR. DEBT BUT A DEBTY TO INCREASE HIS STAKE BY FIFTEEN PERCENT MEMBERS OF MR. LICENSE GROUP WOULD HAVE TO SELL TO HIM
-------- 167

LABEL:_

MINE: THIS MEANS THAT FOR MR. DEBT BUT A DEBTY TO INCREASE HIS STAKE BY FIFTEEN PERCENT MEMBERS OF MR. LICENSE GROUP WOULD HAVE TO SELL TO HIM

decoded1[0]
      SOME FORCES SAID SOME OF THE LESS TRADITIONAL HOLDERS OF GENERALE DOUBT BELGIEFS SHARES COULD BE TEMPERED BY THE PROSPECT OF CAPITAL GAINS
-------- 168

LABEL:_

MINE: SOME FORCES SAID SOME OF THE LESS TRADITIONAL HOLDERS OF GENERALE DOUBT BELGIEFS SHARES COULD BE TEMPERED BY T

decoded1[0]
      AT HOME THE PERMIT SECRETARY OF THE MINISTRY OF COMMUNICATIONS AND INFORMATION HAS REBUEPED THE AMERICAN BUSINESS COUNCIL
-------- 197

LABEL:_

MINE: AT HOME THE PERMIT SECRETARY OF THE MINISTRY OF COMMUNICATIONS AND INFORMATION HAS REBUEPED THE AMERICAN BUSINESS COUNCIL

decoded1[0]
      ANOTHER SPOKESMAN FOR THE MINISTRY HAS REPLIED TO THE FOREIGN COURSE BOND ITS ASSOCIATION OF SALES DEASIAN OF SALES DAYS ASIA
-------- 198

LABEL:_

MINE: ANOTHER SPOKESMAN FOR THE MINISTRY HAS REPLIED TO THE FOREIGN COURSE BOND ITS ASSOCIATION OF SALES DEASIAN OF SALES DAYS ASIA

decoded1[0]
      AND THE PRESS SECRETARY TO THE MINISTER OF COMMUNICATIONS AND INFORMATION HAS RESPONDED TO NEMEROUS CRITICISMS
-------- 199

LABEL:_

MINE: AND THE PRESS SECRETARY TO THE MINISTER OF COMMUNICATIONS AND INFORMATION HAS RESPONDED TO NEMEROUS CRITICISMS

decoded1[0]
      NOBODY KNOWS HOW THE SOUTH AFRICAN GOVERNMENT WILL REACT
-------- 200

LABEL:_

MINE: NOBODY KNOWS HOW THE SOUTH AFRICAN

decoded1[0]
      NICKOJA AND AND THE HOUL EARNED DIFFERENT SEGREES FROM THE OIL SERVICES BUSINESS
-------- 231

LABEL:_

MINE: NICKOJA AND AND THE HOUL EARNED DIFFERENT SEGREES FROM THE OIL SERVICES BUSINESS

decoded1[0]
      AN HOUSE SERVICES WELLS WANT STEER DRAWLED WALMAN DEMOND BUILT OFFSHORED DROILLING ROOMS
-------- 232

LABEL:_

MINE: AN HOUSE SERVICES WELLS WANT STEER DRAWLED WALMAN DEMOND BUILT OFFSHORED DROILLING ROOMS

decoded1[0]
      BUT BOTH COMPANIES SHOULD BENEFIT FROM AN UPTURN IN THE OIL AND NETRAL GAS BUSINESS
-------- 233

LABEL:_

MINE: BUT BOTH COMPANIES SHOULD BENEFIT FROM AN UPTURN IN THE OIL AND NETRAL GAS BUSINESS

decoded1[0]
      THRIFT NET WORSE
-------- 234

LABEL:_

MINE: THRIFT NET WORSE

decoded1[0]
      ASSETS MINOFICILE ABILITIES EXPRESSED AS A PERCENTAGE OF TOTAL LIABILITIES
-------- 235

LABEL:_

MINE: ASSETS MINOFICILE ABILITIES EXPRESSED AS A PERCENTAGE OF TOTAL LIABILITIES

decoded1[0]
      THE NATIONAL AVERAGE WAS FOUR POINT SIX THREE PERC

decoded1[0]
      IN ADDITION AND NUCLEAR FREEZE OWNS OFTEN INCLUDE ARRANGEMENTS THAT REQUIRED IN A CLEAR WEAPONS DEPLOYED A JACENTS OF THE ZONE ON LAND OR AT C. B. POLLED BAT
-------- 261

LABEL:_

MINE: IN ADDITION AND NUCLEAR FREEZE OWNS OFTEN INCLUDE ARRANGEMENTS THAT REQUIRED IN A CLEAR WEAPONS DEPLOYED A JACENTS OF THE ZONE ON LAND OR AT C. B. POLLED BAT

decoded1[0]
      YIELD MANAGEMENT ISN'T ALL BAD FOR CONSUMERS
-------- 262

LABEL:_

MINE: YIELD MANAGEMENT ISN'T ALL BAD FOR CONSUMERS

decoded1[0]
      HOLTELS CAN USE THE TECHNOLOGY TO PREDICT ONE BUSINESS WILL BE UNUSUALLY SLOW AND THUS ONE MORE DISCOUNTING IS NAMED IT
-------- 263

LABEL:_

MINE: HOLTELS CAN USE THE TECHNOLOGY TO PREDICT ONE BUSINESS WILL BE UNUSUALLY SLOW AND THUS ONE MORE DISCOUNTING IS NAMED IT

decoded1[0]
      AND SOME CHAINS SUCH AS HOLIDAY CORPORATION SHARES INCORPORATION AND HIGH AHEAD HOTELS CORPORATION INSIST THEY WILL MAKE PLENNIFF ROOMS AMALABLE AND BARGAIN RATES
-------- 264

LABEL:_

MINE: 